In [6]:
import pandas as pd
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  

In [16]:
import xmltodict


In [27]:
xm = """
<root>
    <items>
        <item><foo> 1 </foo><bar> 2 </bar></item>
        <item><foo> 1 </foo><bar> 2 </bar></item>
        <item><foo> 1 </foo><bar> 2 </bar></item>
    </items>
</root>
""" 

In [37]:
od = xmltodict.parse( xm ) 
items = od['root']['items']['item']
lst = []
for i in items: 
    lst.append(dict( i ))
lst

[{'foo': '1', 'bar': '2'}, {'foo': '1', 'bar': '2'}, {'foo': '1', 'bar': '2'}]

In [39]:
df = pd.DataFrame(lst) 
df

,foo,bar
0,1,2
1,1,2
2,1,2


AttributeError: 'list' object has no attribute 'keys'

,root
items,"{'item': ['1', '2', '3']}"


In [3]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT CONVERT(NVARCHAR(5) , PK_QuestionTypeId) as QTID, code QCODE  FROM fsma_QuestionTypes
""" 
qtypes = pd.read_sql(query,con=conn)  

In [4]:
def processdf(df):
    df=df.fillna('~')
    df=df.loc[ df.ID != '~' ]
    df.ID = df.ID.astype(str)
    df.reset_index(drop=True, inplace=True) 
    df["SORDER"] = df.index 
    df["TYPE"] = df.TYPE.apply(lambda x: replacetype(x) )
    df["QTEXT"] = df.QTEXT.apply(lambda x: x.replace('\'','`') ) 
    df["ID"] = df.ID.apply(lambda x: x.replace('\'','') ) 
    return df

In [5]:
df_mas = pd.DataFrame();
for i in range(1,4):
    df = pd.read_csv(fr'c:\Users\Tim\Documents\1802\s{i}.csv')  
    df['Section'] = i
    df['OFFSET'] = 50 * (i-1)
    df=processdf(df)
    df_mas = pd.concat([df_mas, df], axis=0, ignore_index=True) 
    
df_mas["QPK"] = df_mas.index + 22575 + df_mas.OFFSET
df_mas.drop(columns=['OFFSET'], inplace=True)
df_mas = pd.merge(df_mas , qtypes, left_on='TYPE', right_on='QCODE', how='left',   )
df_mas["DBINSERT"] = "" 
df_mas["ASPX"] = "" 
df_mas.sort_values( by='QPK', inplace=True)
df_mas.reset_index(drop=True, inplace=True)
df_mas

NameError: name 'replacetype' is not defined

In [39]:
ins = ''
aspx = ''
pickseed = 322
for i,r in df_mas.iterrows():
    df_mas.loc[i, 'DBINSERT'] = f"({r.QPK}, @FormName, @PK_QGroup, {r.SORDER}, N'{r.ID}', NULL, NULL, NULL, {r.QTID}, 1, NULL, 0, NULL, NULL, NULL, N'{r.QTEXT}',  NULL),"
    ins = ins + df_mas.loc[i, 'DBINSERT'] + '\n'  
    metric = r.ID.replace('.','_')
    if r.TYPE == 'PICK':
        pickseed = pickseed + 1    
    try:
        s = open(f'C:\\_som\\_src\\{r.QCODE}.aspx', 'r').read()
        s = s.replace('{0}',str(r.QPK)).replace('{1}',str(metric)).replace('{4}',str(metric)+ ' : ' +str(r.QPK)+ '\n\t' +str(r.QTEXT)).replace('{5}',  'METRIC_' +str(metric) )
        s = s.replace('{2}',str(pickseed))
        df_mas.loc[i, 'ASPX'] = s
        aspx = aspx + df_mas.loc[i, 'ASPX'] + '\n\n' 
    except : 
        pass
 
df_mas.to_csv(r"c:\Users\Tim\Documents\1802\BOD_2021_Metrics.csv", index='false')
with open(r'c:\Users\Tim\Documents\1802\DB_Update7.34_BOD_2021_READONLY.sql', 'w') as file:
  file.write(ins)
file.close()
with open(r'c:\Users\Tim\Documents\1802\BOD_2021_ASPX_READONLY.aspx', 'w') as file:
  file.write(aspx)
file.close()

df_mas

,ID,METRIC,QTEXT,TYPE,Section,SORDER,QPK,QTID,QCODE,DBINSERT,ASPX
0,1,Purpose of HVA,What is the purpose of the HVA?,TXT,1,0,22575,9,TXT,"(22575, @FormName, @PK_QGroup, 0, N'1', NULL, ...",<!-- \n 1 : 22575\n\tWhat is the purpose of...
1,2,What Services does the HVA provide,What services does the HVA system provide?,TXT,1,1,22576,9,TXT,"(22576, @FormName, @PK_QGroup, 1, N'2', NULL, ...",<!-- \n 2 : 22576\n\tWhat services does the...
2,2.1,To Whom does the HVA provide services to,To Whom does the HVA provide services to,TXT,1,2,22577,9,TXT,"(22577, @FormName, @PK_QGroup, 2, N'2.1', NULL...",<!-- \n 2_1 : 22577\n\tTo Whom does the HVA...
3,3,Time Since Deployment,What is the approximate time in years since de...,PICK,1,3,22578,17,PICK,"(22578, @FormName, @PK_QGroup, 3, N'3', NULL, ...",<!-- \n 3 : 22578\n\tWhat is the approximat...
4,4,Legacy Identification,Is this system identified as a Legacy* system ...,YNA,1,4,22579,5,YNA,"(22579, @FormName, @PK_QGroup, 4, N'4', NULL, ...",<!-- \n 4 : 22579\n\tIs this system identif...
...,...,...,...,...,...,...,...,...,...,...,...
66,19.5.1,Other Sensitive Data - Volume,Other Sensitive Data - Volume,PICK,3,15,22741,17,PICK,"(22741, @FormName, @PK_QGroup, 15, N'19.5.1', ...",<!-- \n 19_5_1 : 22741\n\tOther Sensitive D...
67,19.5.2,Other Sensitive Data - Type,Other Sensitive Data - Type,TXT,3,16,22742,9,TXT,"(22742, @FormName, @PK_QGroup, 16, N'19.5.2', ...",<!-- \n 19_5_2 : 22742\n\tOther Sensitive D...
68,19.6.1,IP Data - Volume,IP Data - Volume,PICK,3,17,22743,17,PICK,"(22743, @FormName, @PK_QGroup, 17, N'19.6.1', ...",<!-- \n 19_6_1 : 22743\n\tIP Data - Volume\...
69,19.6.2,IP Data - Type,IP Data - Type,TXT,3,18,22744,9,TXT,"(22744, @FormName, @PK_QGroup, 18, N'19.6.2', ...",<!-- \n 19_6_2 : 22744\n\tIP Data - Type\n-...


In [75]:
df_picks = pd.DataFrame(); 
df = pd.read_csv(fr'c:\Users\Tim\Documents\1802\s1.csv')   
df = df.fillna('~')
df=df.loc[ df.TYPE != '~' ] 
df=df.loc[ df.METRIC.str.contains('PL-')  ] 
pickseed = 3980
sorder=0
METRIC=df.iloc[0]['METRIC']
for i,r in df.iterrows():
    CodeValue = re.sub('[AEIOUY]', '', r.TYPE.upper())
    CodeValue = re.sub('[^A-Z0-9]', '', CodeValue)  
    if METRIC != r.METRIC:
        sorder=0
        METRIC=r.METRIC 
    pickseed = pickseed + 1 
    sorder = sorder + 1 
    TYPE= re.sub('\s\s', '', r.TYPE)  
    df.loc[i, 'PK_Picklist'] = f"({pickseed},@PKT, N'{CodeValue[:4]+CodeValue[-4:]}','{TYPE}',{sorder}, GETDATE(), 1),"
    print(df.loc[i, 'PK_Picklist']) 
df.head(59)

(3981,@PKT, N'5RS5RS','· < 5 years',1, GETDATE(), 1),
(3982,@PKT, N'610R10RS','· > 6 – 10 years',2, GETDATE(), 1),
(3983,@PKT, N'101515RS','· > 10 – 15 years',3, GETDATE(), 1),
(3984,@PKT, N'152020RS','· > 15 – 20 years',4, GETDATE(), 1),
(3985,@PKT, N'203030RS','· > 20 – 30 years',5, GETDATE(), 1),
(3986,@PKT, N'304040RS','· > 30 – 40 years',6, GETDATE(), 1),
(3987,@PKT, N'40RS40RS','· > 40 years',7, GETDATE(), 1),
(3988,@PKT, N'LGCCMMPS','Legacy codebase (COBOL, MUMPS)',1, GETDATE(), 1),
(3989,@PKT, N'LCKFNTTN','Lack of or poor documentation',2, GETDATE(), 1),
(3990,@PKT, N'LGCNRCTR','Legacy infrastructure',3, GETDATE(), 1),
(3991,@PKT, N'DHCNRCTR','Ad-hoc infrastructure ',4, GETDATE(), 1),
(3992,@PKT, N'HRDWPPRT','Hardware or software with no vendor/manufacturer support',5, GETDATE(), 1),
(3993,@PKT, N'TDTDVNTR','Outdated or inaccurate inventory',6, GETDATE(), 1),
(3994,@PKT, N'TCHNLGTP','Technology Type ',7, GETDATE(), 1),
(3995,@PKT, N'SSTMRNZD','System Last Modernized',8, GETDATE

,ID,METRIC,QTEXT,TYPE,PK_Picklist
4,~,PL-APPDPLY,~,· < 5 years,"(3981,@PKT, N'5RS5RS','· < 5 years',1, GETDATE..."
5,~,PL-APPDPLY,~,· > 6 – 10 years,"(3982,@PKT, N'610R10RS','· > 6 – 10 years',2, ..."
6,~,PL-APPDPLY,~,· > 10 – 15 years,"(3983,@PKT, N'101515RS','· > 10 – 15 years',3,..."
7,~,PL-APPDPLY,~,· > 15 – 20 years,"(3984,@PKT, N'152020RS','· > 15 – 20 years',4,..."
8,~,PL-APPDPLY,~,· > 20 – 30 years,"(3985,@PKT, N'203030RS','· > 20 – 30 years',5,..."
9,~,PL-APPDPLY,~,· > 30 – 40 years,"(3986,@PKT, N'304040RS','· > 30 – 40 years',6,..."
10,~,PL-APPDPLY,~,· > 40 years,"(3987,@PKT, N'40RS40RS','· > 40 years',7, GETD..."
26,~,PL-LGYCHR,~,"Legacy codebase (COBOL, MUMPS)","(3988,@PKT, N'LGCCMMPS','Legacy codebase (COBO..."
27,~,PL-LGYCHR,~,Lack of or poor documentation,"(3989,@PKT, N'LCKFNTTN','Lack of or poor docum..."
28,~,PL-LGYCHR,~,Legacy infrastructure,"(3990,@PKT, N'LGCNRCTR','Legacy infrastructure..."


In [ ]:
 df = pd.read_csv(fr'c:\Users\Tim\Documents\1802\BOD_2021_Metrics.csv')  

In [1]:
query = """\
SELECT PK_PickList, PK_PickListType, DisplayValue, CodeValue, SortPos, Description, UsageField FROM vwPickLists
WHERE PK_PickListType BETWEEN 153 AND 162
"""
dfpicks = pd.read_sql(query,con=conn) 
UsageField=dfpicks.iloc[0]['UsageField']
s=f"\n"
s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[0]['Description']}-{dfpicks.iloc[0]['PK_PickListType']}\n"
for i,r in dfpicks.iterrows():
    if UsageField != dfpicks.iloc[i]['UsageField']:
        UsageField=dfpicks.iloc[i]['UsageField']
        s = s + f"End Enum\n"
        s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[i]['Description']}-{dfpicks.iloc[i]['PK_PickListType']}\n"
    s = s + f"\t{r.CodeValue} = {str(r.PK_PickList)} \t'{str(r.DisplayValue)}\n" 
s=s+"End Enum\n"
print(s)

NameError: name 'pd' is not defined